#Librerías
Son librerias unicamente de los algoritmos

In [5]:
# Fernet
!pip install cryptography
# ARC4
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.7 MB/s eta 0:00:00


In [6]:
# Fernet
from cryptography.fernet import Fernet

In [7]:
# ARC4
import Crypto 
from Crypto.Cipher import ARC4
import secrets

In [8]:
# DSA
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [9]:
# HKDF
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

In [10]:
# SHA256
import hashlib

#Algoritmos Simétricos
##1. Fernet

In [ ]:
# Generamos una clave
clave = Fernet.generate_key()

# Creamos la instancia de Fernet
# Parametros: key: clave generada
f = Fernet(clave)

# Encriptamos el mensaje
# utilizando el método "encrypt"
mensaje = bytes(input(), 'utf-8')
token = f.encrypt(mensaje)

# Mostramos el token del mensaje
print('Token:', token.decode())

hoho
Token: gAAAAABkd2ASZWmyJF95L6g7qTVsiRFfutlBSV4mCOXCpdyD3bg_JWcPbqYKMT0w4SELdI_hynLmPxMSalqI_ZzRTFb4JLd_Ww==


In [ ]:
# Podemos descifrar el mensaje utilizando el método "decrypt".
des = f.decrypt(token)
# decode() sirve para volver a formato string
print(des.decode())

hoho


##2. ARC4

In [ ]:
def rc4(texto):
    clave = secrets.token_bytes(16)  # Genera una clave aleatoria de 16 bytes
    cipher = ARC4.new(clave)
    texto_cifrado = cipher.encrypt(texto)
    return clave, texto_cifrado

In [ ]:
texto_plano = bytes(input(), 'utf-8')
key, ciphertext = rc4(texto_plano)
print("Clave: ", key)
print("Texto cifrado: ", ciphertext)

hola mundo
Clave:  b'G\xad=\xa0\x96\xae\x85\x9f(\x97t2P\xc4\x87\xb1'
Texto cifrado:  b'\xa9e I\xd0f\xb1\xa2"\x92'


In [ ]:
cipher = ARC4.new(key)
texto_descifrado = cipher.decrypt(ciphertext).decode()
print("Texto descifrado:", texto_descifrado)

Texto descifrado: hola mundo


#Algoritmo Asimétrico
##DSA (Digital Sign Algorithm)

In [ ]:
# Generar una clave privada DSA
clave_privada = dsa.generate_private_key(key_size=1024)

# Obtener la clave pública correspondiente
clave_publica = clave_privada.public_key()

# Generamos una clave privada de otro algoritmo y obtenemos su clave publica
private_key = rsa.generate_private_key(
     public_exponent=65537,
     key_size=2048,
     backend=default_backend()
)
public_key = private_key.public_key()

# Serializar la clave pública en formato PEM
clave_publica_pem = clave_publica.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Mensaje a firmar
mensaje = bytes(input(), 'utf-8')

# Firmar el mensaje utilizando la clave privada
firma = clave_privada.sign(
    mensaje,
    hashes.SHA256()
)

# Verificar la firma utilizando la clave pública
try:
    clave_publica.verify(
      firma,
      mensaje,
      hashes.SHA256()
    ) 
    print("La firma es valida. Clave publica: ", clave_publica)
except Exception:
  print("La firma no es valida")

# Verificar la firma utilizando la clave pública que no es correcta
try:
    public_key.verify(
      firma,
      mensaje,
      hashes.SHA256()
    ) 
    print("La firma es valida.")
except Exception:
    print("La firma no es valida. Clave publica: ", public_key)

print("Mensaje original:  ", mensaje.decode())

hola mundo
La firma es valida. Clave publica:  <cryptography.hazmat.backends.openssl.dsa._DSAPublicKey object at 0x7fd80f0ec9d0>
La firma no es valida. Clave publica:  <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7fd80f0efa00>
Mensaje original:   hola mundo


##Algoritmo Adicional Asimétrico
##HKDF

In [ ]:
#Generar las claves para Alice
alice_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
alice_public_key = alice_private_key.public_key()

#Generar las claves para Bob
bob_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
bob_public_key = bob_private_key.public_key()

#Intercambio de claves
alice_shared_key = alice_private_key.exchange(ec.ECDH(), bob_public_key)
bob_shared_key = bob_private_key.exchange(ec.ECDH(), alice_public_key)

#Verificación de que las claves compartidas sean iguales
assert alice_shared_key == bob_shared_key

#Derivar una clave de sesión utilizando HKDF
hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'session_key',
    backend=default_backend()
)
session_key = hkdf.derive(alice_shared_key)

#Cifrado del mensaje de Alice a Bob
plaintext = bytes(input(), 'utf-8')
nonce = os.urandom(16)
cipher = Cipher(algorithms.AES(session_key), modes.CTR(nonce), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext) + encryptor.finalize()

#Descifrado del mensaje de Bob
decryptor = cipher.decryptor()
decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()

#Verificación de que el mensaje descifrado sea igual al original
if decrypted_text == plaintext:
  print("Descifrado correcto")

#Imprimir el mensaje cifrado y descifrado
print("Mensaje cifrado: ", ciphertext.hex())
print("Mensaje descifrado: ", decrypted_text.decode())

hola
Descifrado correcto
Mensaje cifrado:  06bfbb0d
Mensaje descifrado:  hola


#Función Hash
##SHA256

In [15]:
def calcular_sha256(texto): # Recibimos texto
    # Creamos un objeto de hash SHA256
    sha256 = hashlib.sha256()

    # Actualizamos el hash con los bytes del texto
    sha256.update(texto)

    # Obtenemos el hash final en formato hexadecimal
    hash_sha256 = sha256.hexdigest()

    return hash_sha256

In [16]:
texto = bytes(input(), 'utf-8')
hash_resultado = calcular_sha256(texto)
print("Hash SHA256:", hash_resultado)

hola
Hash SHA256: b221d9dbb083a7f33428d7c2a3c3198ae925614d70210e28716ccaa7cd4ddb79


##Linea para generar claves aleatorias

In [ ]:
import os

claveEj = os.urandom(32) # Clave aleatoria en formato Byte de 32 bytes
print('Clave: ', clave)

Clave:  b'BfLA8Q0VawBAqNm3ytY2JXA8gsHt83KJRhW6x6BhZ-Y='


#Funcion para medir el tiempo
##Opcion 1 
Usando time.time()

In [ ]:
import time

In [ ]:
inicio = time.time()
#aqui va el metodo
fin = time.time()

duracion = fin - inicio

print("Tiempo de ejecución:", duracion, "segundos")

Tiempo de ejecución: 3.409385681152344e-05 segundos


#Opcion 2
Usando timeit

In [ ]:
import timeit

In [ ]:
# Para metodos sin parametros
def mi_metodo():
    # Código de tu método
    pass

# Medir el tiempo de ejecución
tiempo = timeit.timeit(mi_metodo, number=1)

print("Tiempo de ejecución:", tiempo, "segundos")

Tiempo de ejecución: 1.522999994563179e-06 segundos


###Si el metodo contiene parametros, se hace:

In [ ]:
from functools import partial

In [ ]:
# Definir tu método con parámetros
def mi_metodo(a, b):
    a + b

# Crear una función parcial con los argumentos fijos
mi_metodo_parcial = partial(mi_metodo, 1, 2)  # Reemplaza arg1 y arg2 con tus valores

# Medir el tiempo de ejecución
tiempo = timeit.timeit(mi_metodo_parcial, number=1)

print("Tiempo de ejecución:", tiempo, "segundos")

Tiempo de ejecución: 1.4859999737382168e-06 segundos


 La función partial() se utiliza para crear una nueva función que tiene los argumentos arg1 y arg2 ya fijados.

#Generador de texto
Usando lorem_text

In [23]:
!pip install lorem-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from lorem_text import lorem

# Generar un texto de estilo "Lorem Ipsum" con un número específico de palabras
def generar_lorem(num_palabras):
    texto = lorem.words(num_palabras)
    return texto

num_palabras_deseadas = 1000
texto_lorem = generar_lorem(num_palabras_deseadas)
print(texto_lorem)

consequatur ut est nobis quis voluptate architecto rem provident accusantium alias molestiae eligendi cupiditate cum vel magnam id doloremque expedita eveniet hic magni aut impedit nemo quaerat quae nostrum praesentium velit modi placeat facere assumenda nisi eum in doloribus et adipisci amet culpa quas saepe eius nam sit fugiat atque deserunt dolore beatae incidunt odio debitis repellat aliquid ratione suscipit officia illum ullam sunt soluta distinctio numquam tempora explicabo aliquam ea quidem accusamus laboriosam rerum consequuntur reprehenderit voluptatem earum maiores natus minus obcaecati omnis quia quibusdam ab dolores quam fuga facilis possimus quasi sint ipsum nihil eaque tenetur fugit eos voluptates perferendis qui excepturi quod iure dolorum corporis nulla ipsam delectus error animi porro voluptatibus commodi sed nesciunt iste labore officiis asperiores illo sequi neque pariatur voluptas maxime laudantium aspernatur non harum a molestias cumque consectetur libero tempore c